In [1]:
# Instalar librerías necesarias
!pip install minisom
!pip install mealpy

  Preparing metadata (setup.py) ... done
  Created wheel for minisom: filename=MiniSom-2.3.5-py3-none-any.whl size=12031 sha256=9f566e28e91b9bca23dfc31249a9d0615baa06ee926c81d732f32f1b5e44929d
  Stored in directory: /root/.cache/pip/wheels/19/db/95/5e53bc2b88a328217fdf9f2886cafbe86b0df274f4b601f572
Successfully built minisom
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.9/397.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 46.8 MB/s eta 0:00:00


In [3]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler
from minisom import MiniSom
from mealpy.swarm_based.GWO import OriginalGWO
from mealpy.utils.space import FloatVar, IntegerVar

# Cargar y normalizar datos
iris = load_iris()
X = iris.data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Función objetivo basada en RMSE
def fitness_function(solution):
    iterations = int(solution[0])
    learning_rate = float(solution[1])
    sigma = float(solution[2])
    map_size = int(round(solution[3]))
    map_size = max(2, map_size)

    som = MiniSom(x=map_size, y=map_size, input_len=X_scaled.shape[1],
                  sigma=sigma, learning_rate=learning_rate, random_seed=42)
    som.random_weights_init(X_scaled)
    som.train(X_scaled, iterations, verbose=False)

    errors = []
    for x in X_scaled:
        bmu = som.get_weights()[som.winner(x)]
        error = np.linalg.norm(x - bmu)
        errors.append(error ** 2)

    rmse = np.sqrt(np.mean(errors))
    return rmse

# Definir el espacio de búsqueda
problem = {
    "obj_func": fitness_function,
    "bounds": [
        IntegerVar(100, 1000),   # iteraciones
        FloatVar(0.01, 1.0),     # learning rate
        FloatVar(0.1, 3.0),      # sigma
        IntegerVar(4, 10)        # tamaño del mapa
    ],
    "minmax": "min",
    "verbose": True
}

# Ejecutar algoritmo GWO
model = OriginalGWO(epoch=30, pop_size=10)
solution = model.solve(problem)

# Extraer resultados
best_position = solution.solution
best_fitness = solution.target.objectives[0]

# Imprimir resultados
print("\n Mejores hiperparámetros encontrados:")
print(f"Iteraciones         : {int(best_position[0])}")
print(f"Tasa de aprendizaje : {best_position[1]:.4f}")
print(f"Sigma               : {best_position[2]:.4f}")
print(f"Tamaño del mapa     : {int(round(best_position[3]))}x{int(round(best_position[3]))}")
print(f"RMSE mínimo         : {best_fitness:.6f}")


INFO:mealpy.swarm_based.GWO.OriginalGWO:OriginalGWO(epoch=30, pop_size=10)
INFO:mealpy.swarm_based.GWO.OriginalGWO:>>>Problem: P, Epoch: 1, Current best: 0.039644409319962964, Global best: 0.039644409319962964, Runtime: 0.52663 seconds
INFO:mealpy.swarm_based.GWO.OriginalGWO:>>>Problem: P, Epoch: 2, Current best: 0.03958198203531734, Global best: 0.03958198203531734, Runtime: 0.54781 seconds
INFO:mealpy.swarm_based.GWO.OriginalGWO:>>>Problem: P, Epoch: 3, Current best: 0.03958198203531734, Global best: 0.03958198203531734, Runtime: 0.61955 seconds
INFO:mealpy.swarm_based.GWO.OriginalGWO:>>>Problem: P, Epoch: 4, Current best: 0.038562739865319606, Global best: 0.038562739865319606, Runtime: 0.64415 seconds
INFO:mealpy.swarm_based.GWO.OriginalGWO:>>>Problem: P, Epoch: 5, Current best: 0.038562739865319606, Global best: 0.038562739865319606, Runtime: 0.53359 seconds
INFO:mealpy.swarm_based.GWO.OriginalGWO:>>>Problem: P, Epoch: 6, Current best: 0.03808265717623301, Global best: 0.038082657


 Mejores hiperparámetros encontrados:
Iteraciones         : 995
Tasa de aprendizaje : 0.9953
Sigma               : 0.3175
Tamaño del mapa     : 10x10
RMSE mínimo         : 0.037786


In [4]:
# Crear y entrenar SOM con esos parámetros
som = MiniSom(x=10, y=10, input_len=X_scaled.shape[1],
              sigma=0.3175, learning_rate=0.9953, random_seed=42)
som.random_weights_init(X_scaled)
som.train(X_scaled, 995, verbose=False)

# Calcular RMSE en el conjunto de datos
errors = []
for x in X_scaled:
    bmu = som.get_weights()[som.winner(x)]
    error = np.linalg.norm(x - bmu)
    errors.append(error ** 2)

rmse = np.sqrt(np.mean(errors))

print(f"RMSE: {rmse:.6f}")

RMSE: 0.037786
